In [1]:
#import
import numpy as np

import torch
from torch.autograd import Variable
from torch import optim

In [2]:
#load in processed data from CollegeBasketballPredictor.ipynb
def load_marchmadness():
    #features from training, test, validation sets
    featurestrainingData = torch.load('featurestraining_pytorch.pt')
    featurestestData = torch.load('featurestest_pytorch.pt')
    featuresvalidationData = torch.load('featuresvalidation_pytorch.pt')
    
    #labels from training, test, validation sets
    labelstrainingData = torch.load('labelstraining_pytorch.pt')
    labelstestData = torch.load('labelstest_pytorch.pt')
    labelsvalidationData = torch.load('labelsvalidation_pytorch.pt')
    
    return featurestrainingData, labelstrainingData, featurestestData, labelstestData, featuresvalidationData, labelsvalidationData

In [3]:
def build_model(input_dim, output_dim):
    # We don't need the softmax layer here since CrossEntropyLoss already
    # uses it internally.
    model = torch.nn.Sequential()
    model.add_module("linear",
                     torch.nn.Linear(input_dim, output_dim, bias=False))
    return model


In [4]:
def train(model, loss, optimizer, x, y):
    # Reset gradient
    optimizer.zero_grad()

    # Forward
    fx = model.forward(x)
    output = loss.forward(fx, y)

    # Backward
    output.backward()

    # Update parameters
    optimizer.step()

    return output.item()


In [5]:
def predict(model, x):
    output = model.forward(x)
    return output.data.numpy().argmax(axis=1)

In [6]:
def main():
    torch.manual_seed(42)
    #X is data Y is label
    trX, trY, teX, teY, valX, valY = load_marchmadness()
    
    n_examples, n_features = list(trX.size())[0], list(trX.size())[1]
    n_classes = 2
    model = build_model(n_features, n_classes)
    loss = torch.nn.CrossEntropyLoss(reduction='elementwise_mean')
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    batch_size = 100

    for i in range(100):
        cost = 0.
        num_batches = n_examples // batch_size
        for k in range(num_batches):
            start, end = k * batch_size, (k + 1) * batch_size
            cost += train(model, loss, optimizer,
                          trX[start:end], trY[start:end])
        predY = predict(model, teX)
        print("Epoch %d, cost = %f, acc = %.2f%%"
              #.cpu().numpy() turns it back into a numpy array
              % (i + 1, cost / num_batches, 100. * np.mean(predY == teY.cpu().numpy())))

In [7]:
main()

/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:16: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")


Epoch 1, cost = 18680.338339, acc = 49.39%
Epoch 2, cost = 15630.104081, acc = 63.61%
Epoch 3, cost = 15270.767256, acc = 50.61%
Epoch 4, cost = 11571.682060, acc = 70.85%
Epoch 5, cost = 14266.887099, acc = 50.79%
Epoch 6, cost = 3834.958311, acc = 66.41%
Epoch 7, cost = 1970.878921, acc = 49.54%
Epoch 8, cost = 819.781875, acc = 82.79%
Epoch 9, cost = 671.177388, acc = 81.71%
Epoch 10, cost = 701.495142, acc = 65.75%
Epoch 11, cost = 729.243179, acc = 90.00%
Epoch 12, cost = 550.017137, acc = 76.79%
Epoch 13, cost = 539.084890, acc = 58.46%
Epoch 14, cost = 678.705344, acc = 49.58%
Epoch 15, cost = 528.027367, acc = 75.04%
Epoch 16, cost = 428.361453, acc = 71.92%
Epoch 17, cost = 614.781588, acc = 62.80%
Epoch 18, cost = 473.621343, acc = 92.76%
Epoch 19, cost = 803.190519, acc = 49.39%
Epoch 20, cost = 2312.080110, acc = 86.66%
Epoch 21, cost = 353.466532, acc = 93.46%
Epoch 22, cost = 280.150261, acc = 90.88%
Epoch 23, cost = 247.669779, acc = 93.02%
Epoch 24, cost = 259.891731, a

KeyboardInterrupt: 